In [3]:
import os
import pickle

results_dir = 'results'
problem = '25fv47'

lp_fn = os.path.join(results_dir, problem + '_lp.p')
sd_fn = os.path.join(results_dir, problem + '_sd.p')

with open(lp_fn, 'rb') as f:
    lp_results = pickle.load(f)
with open(sd_fn, 'rb') as f:
    sd_results = pickle.load(f)
    
print(lp_results['obj'])
print(sd_results['obj'])


5501.84588828669
5501.845088066554
